In [1]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.5/637.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00


In [2]:
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import os
import random
import shutil
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import shutil
import mlflow

In [3]:
if os.path.exists('data'):
  shutil.rmtree('data')

In [4]:
!kaggle datasets download -d ghostbat101/lung-x-ray-image-clinical-text-dataset -p data

Dataset URL: https://www.kaggle.com/datasets/ghostbat101/lung-x-ray-image-clinical-text-dataset
License(s): MIT
 98% 1.29G/1.33G [00:05<00:00, 293MB/s]
100% 1.33G/1.33G [00:05<00:00, 274MB/s]


In [5]:
!unzip -n data/lung-x-ray-image-clinical-text-dataset.zip -d data

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45915.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_4592.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45950.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45951.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45952.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45953.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45956.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45957.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45961.jpg  
  inflating: data/Merged Data (image+text)/Obstructive Pulmonary Diseases/image_45964.jpg  
  i

On remplace les espaces par _ dans les noms des dossiers et fichiers

In [6]:
def replace_spaces_with_underscores(directory):
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in dirs:
            new_name = name.replace(' ', '_')
            if new_name != name:
                old_path = os.path.join(root, name)
                new_path = os.path.join(root, new_name)
                shutil.move(old_path, new_path)
                print(f"Renamed: {old_path} -> {new_path}")

replace_spaces_with_underscores('data')


Renamed: data/Merged Data (image+text)/Lower Density -> data/Merged Data (image+text)/Lower_Density
Renamed: data/Merged Data (image+text)/Chest Changes -> data/Merged Data (image+text)/Chest_Changes
Renamed: data/Merged Data (image+text)/Encapsulated Lesions -> data/Merged Data (image+text)/Encapsulated_Lesions
Renamed: data/Merged Data (image+text)/Higher Density -> data/Merged Data (image+text)/Higher_Density
Renamed: data/Merged Data (image+text)/Mediastinal Changes -> data/Merged Data (image+text)/Mediastinal_Changes
Renamed: data/Merged Data (image+text)/Obstructive Pulmonary Diseases -> data/Merged Data (image+text)/Obstructive_Pulmonary_Diseases
Renamed: data/Merged Data (image+text)/Degenerative Infectious Diseases -> data/Merged Data (image+text)/Degenerative_Infectious_Diseases
Renamed: data/Main dataset/Lower Density -> data/Main dataset/Lower_Density
Renamed: data/Main dataset/Chest Changes -> data/Main dataset/Chest_Changes
Renamed: data/Main dataset/Encapsulated Lesions 

On met de côté 20 % des données pour le test

In [7]:
# Chemin vers le dataset principal
source_dir = "data/Main_dataset"

# Chemin vers le dossier où les 20% seront stockés
test_dir = "data/Test_dataset"

# Fraction des données à extraire
test_split = 0.2

# Définir un seed pour rendre le tirage aléatoire reproductible
random.seed(42)

# Créer le dossier de sortie
os.makedirs(test_dir, exist_ok=True)

# Parcourir chaque classe (dossier dans le dataset principal)
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if os.path.isdir(class_path):  # Vérifier que c'est un dossier
        # Lister tous les fichiers de la classe
        images = os.listdir(class_path)

        # Mélanger les fichiers aléatoirement
        random.shuffle(images)

        # Calculer le nombre de fichiers à déplacer
        num_test_files = int(len(images) * test_split)
        test_files = images[:num_test_files]

        # Créer un dossier pour cette classe dans le dossier de test
        class_test_dir = os.path.join(test_dir, class_name)
        os.makedirs(class_test_dir, exist_ok=True)

        # Déplacer les fichiers sélectionnés
        for img in test_files:
            src_path = os.path.join(class_path, img)
            dest_path = os.path.join(class_test_dir, img)
            shutil.move(src_path, dest_path)

print(f"20% des données ont été déplacées vers le dossier '{test_dir}'")

20% des données ont été déplacées vers le dossier 'data/Test_dataset'


# Creation d'un sample avec 20 % des données : Main dataset Sample

In [8]:
# Chemin vers le dataset principal
source_dir = "data/Main_dataset"

# Chemin vers le dossier où les 20% seront copiés
test_dir = "data/Main_dataset_Sample"

# Fraction des données à extraire
main_sample = 0.2

# Définir un seed pour rendre le tirage aléatoire reproductible
random.seed(42)

# Créer le dossier de sortie pour le dataset de test
os.makedirs(test_dir, exist_ok=True)

# Parcourir chaque classe (dossier dans le dataset principal)
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)

    if os.path.isdir(class_path):  # Vérifier que c'est un dossier
        # Lister tous les fichiers de la classe
        images = os.listdir(class_path)

        # Mélanger les fichiers aléatoirement
        random.shuffle(images)

        # Calculer le nombre de fichiers à dupliquer
        num_sample_files = int(len(images) * main_sample)  # Correction du nom de la variable
        sample_files = images[:num_sample_files]

        # Créer un dossier pour cette classe dans le dossier de test
        class_sample_dir = os.path.join(test_dir, class_name)
        os.makedirs(class_sample_dir, exist_ok=True)  # Correction du nom du dossier

        # Dupliquer les fichiers sélectionnés dans le dossier de test
        for img in sample_files:  # Correction de la variable test_files en sample_files
            src_path = os.path.join(class_path, img)
            dest_path = os.path.join(class_sample_dir, img)
            shutil.copy(src_path, dest_path)  # Utilisation de shutil.copy() pour dupliquer

print("Les 20% des données ont été dupliquées dans le répertoire Main dataset Sample.")

Les 20% des données ont été dupliquées dans le répertoire Main dataset Sample.


# Transformation en binaire : main dataset sample binaire et en multiclasse malade : main dataset sample multiclasse

In [9]:
# Chemin vers le dataset principal
source_dir = "data/Main_dataset_Sample"

# Chemin vers le nouveau dataset binaire
binary_sample_dir = "data/Main_dataset_Sample_Binaire"


# Créer les répertoires de sortie pour les classes "Normal" et "Malades"
os.makedirs(binary_sample_dir, exist_ok=True)
normal_dir = os.path.join(binary_sample_dir, "Normal")
malades_dir = os.path.join(binary_sample_dir, "Malades")
os.makedirs(normal_dir, exist_ok=True)
os.makedirs(malades_dir, exist_ok=True)

# Liste des classes malades
malades_classes = ['Chest_Changes', 'Degenerative_Infectious_Diseases', 'Encapsulated_Lesions',
                    'Higher_Density', 'Lower_Density', 'Mediastinal_Changes', 'Obstructive_Pulmonary_Diseases']

# Parcourir chaque classe (dossier dans le dataset principal)
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)

    if os.path.isdir(class_path):  # Vérifier que c'est un dossier
        # Lister tous les fichiers de la classe
        images = os.listdir(class_path)

        # Mélanger les fichiers aléatoirement
        random.shuffle(images)

        # Calculer le nombre de fichiers à dupliquer
        num_sample_files = int(len(images) * main_sample)
        sample_files = images[:num_sample_files]

        if class_name == 'Normal':
            # Créer le dossier pour la classe "Normal"
            for img in sample_files:
                src_path = os.path.join(class_path, img)
                dest_path = os.path.join(normal_dir, img)
                shutil.copy(src_path, dest_path)  # Copier l'image dans le dossier "Normal"

        elif class_name in malades_classes:
            # Créer le dossier pour les classes malades
            for img in sample_files:
                src_path = os.path.join(class_path, img)
                dest_path = os.path.join(malades_dir, img)
                shutil.copy(src_path, dest_path)  # Copier l'image dans le dossier "Malades"

print("Les données ont été dupliquées dans le répertoire Main dataset Sample Binaire avec les classes 'Normal' et 'Malades'.")

Les données ont été dupliquées dans le répertoire Main dataset Sample Binaire avec les classes 'Normal' et 'Malades'.


In [10]:

# Chemin vers le dataset principal
source_dir = "data/Main_dataset_Sample"

# Chemin vers le nouveau dataset binaire
binary_sample_dir = "data/Main_dataset_Sample_Binaire"

# Chemin vers le nouveau dataset multiclasse
multiclass_sample_dir = "data/Main_dataset_Sample_Multiclasse"

# Créer les répertoires de sortie pour les classes "Normal" et "Malades"
os.makedirs(binary_sample_dir, exist_ok=True)
normal_dir = os.path.join(binary_sample_dir, "Normal")
malades_dir = os.path.join(binary_sample_dir, "Malades")
os.makedirs(normal_dir, exist_ok=True)
os.makedirs(malades_dir, exist_ok=True)

# Créer le répertoire de sortie pour le dataset multiclasse
os.makedirs(multiclass_sample_dir, exist_ok=True)

# Liste des classes malades
malades_classes = ['Chest_Changes', 'Degenerative_Infectious_Diseases', 'Encapsulated_Lesions',
                   'Higher_Density', 'Lower_Density', 'Mediastinal_Changes', 'Obstructive_Pulmonary_Diseases']

# Parcourir chaque classe (dossier dans le dataset principal)
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)

    if os.path.isdir(class_path):  # Vérifier que c'est un dossier
        # Lister tous les fichiers de la classe
        images = os.listdir(class_path)

        # Mélanger les fichiers aléatoirement
        random.shuffle(images)

        # Calculer le nombre de fichiers à dupliquer
        num_sample_files = int(len(images) * main_sample)
        sample_files = images[:num_sample_files]

        # Gestion pour le dataset binaire
        if class_name == 'Normal':
            for img in sample_files:
                src_path = os.path.join(class_path, img)
                dest_path = os.path.join(normal_dir, img)
                shutil.copy(src_path, dest_path)  # Copier l'image dans le dossier "Normal"

        elif class_name in malades_classes:
            for img in sample_files:
                src_path = os.path.join(class_path, img)
                dest_path = os.path.join(malades_dir, img)
                shutil.copy(src_path, dest_path)  # Copier l'image dans le dossier "Malades"

        # Gestion pour le dataset multiclasse
        multiclass_dir = os.path.join(multiclass_sample_dir, class_name)
        os.makedirs(multiclass_dir, exist_ok=True)  # Créer un sous-dossier pour chaque classe
        for img in sample_files:
            src_path = os.path.join(class_path, img)
            dest_path = os.path.join(multiclass_dir, img)
            shutil.copy(src_path, dest_path)  # Copier l'image dans son dossier de classe

print("Les données ont été dupliquées :")
print("- Dans le répertoire Main_dataset_Sample_Binaire avec les classes 'Normal' et 'Malades'.")
print("- Dans le répertoire Main_dataset_Sample_Multiclasse avec les sous-dossiers pour chaque classe.")


Les données ont été dupliquées :
- Dans le répertoire Main_dataset_Sample_Binaire avec les classes 'Normal' et 'Malades'.
- Dans le répertoire Main_dataset_Sample_Multiclasse avec les sous-dossiers pour chaque classe.


In [11]:
ratio_malade = 0.5

# Chemin vers le dataset principal
source_dir = "data/Main_dataset"

# Chemin vers le nouveau dataset binaire
binary_sample_dir = "data/Main_dataset_Sample_Binaire_With_Ratio"

# Créer les répertoires de sortie pour les classes "Normal" et "Malades"
os.makedirs(binary_sample_dir, exist_ok=True)
normal_dir = os.path.join(binary_sample_dir, "Normal")
malades_dir = os.path.join(binary_sample_dir, "Malades")
os.makedirs(normal_dir, exist_ok=True)
os.makedirs(malades_dir, exist_ok=True)

# Créer le répertoire de sortie pour le dataset multiclasse
os.makedirs(multiclass_sample_dir, exist_ok=True)

# Liste des classes malades
malades_classes = ['Chest_Changes', 'Degenerative_Infectious_Diseases', 'Encapsulated_Lesions',
                   'Higher_Density', 'Lower_Density', 'Mediastinal_Changes', 'Obstructive_Pulmonary_Diseases']

normal_classes = ['Normal']

normal_image_count = int((1- ratio_malade) * len(os.listdir(os.path.join(source_dir, normal_classes[0]))))
print(normal_image_count)
malades_image_count = int(ratio_malade * len(os.listdir(os.path.join(source_dir, normal_classes[0]))))
print(malades_image_count)

for class_name in normal_classes:
  class_path = os.path.join(source_dir, class_name)
  images = os.listdir(class_path)
  random.shuffle(images)
  num_sample_files = int(normal_image_count / len(normal_classes))
  sample_files = images[:num_sample_files]
  for img in sample_files:
    src_path = os.path.join(class_path, img)
    dest_path = os.path.join(normal_dir, img)
    shutil.copy(src_path, dest_path)

for class_name in malades_classes:
  class_path = os.path.join(source_dir, class_name)
  images = os.listdir(class_path)
  random.shuffle(images)
  num_sample_files = int(malades_image_count / len(malades_classes))
  sample_files = images[:num_sample_files]
  for img in sample_files:
    src_path = os.path.join(class_path, img)
    dest_path = os.path.join(malades_dir, img)
    shutil.copy(src_path, dest_path)

print("Les données ont été dupliquées :")
print(f"- Dans le répertoire {binary_sample_dir} avec les classes 'Normal' et 'Malades'.")



4000
4000
Les données ont été dupliquées :
- Dans le répertoire data/Main_dataset_Sample_Binaire_With_Ratio avec les classes 'Normal' et 'Malades'.


In [12]:
def dupliquer_dossier(source_dir, destination_dir):
    # Vérifier si le dossier source existe
    if not os.path.exists(source_dir):
        print(f"Le dossier source {source_dir} n'existe pas.")
        return

    # Vérifier si le dossier de destination existe, sinon le créer
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Copier le contenu du dossier source vers le dossier de destination
    try:
        shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
        print(f"Le dossier {source_dir} a été dupliqué vers {destination_dir}")
    except Exception as e:
        print(f"Erreur lors de la duplication : {e}")

In [13]:
# Chemin vers le dataset principal
source_dir = "data/Main_dataset"

# Chemin vers le nouveau dataset binaire
seven_classes_dir = "data/Main_dataset_7_classes"

# Copie tous les fichiers
dupliquer_dossier(source_dir, seven_classes_dir)

# Supprime la classe Normal
shutil.rmtree(os.path.join(seven_classes_dir, "Normal"))

Le dossier data/Main_dataset a été dupliqué vers data/Main_dataset_7_classes


# Démarrage du tracking MLFLOW

In [14]:
MLFLOW_SERVER_URI = 'https://david-rem-jedha-final-project-mlops.hf.space'
EXPERIMENT_NAME = 'multi' # 'binary' ou 'multi'
TRAINER = 'david' # Le prénom de la personne qui a exécuté l'entrainement
MODEL_TYPE = 'VGG19' # Le type de modèle utilisé

mlflow.set_tracking_uri(MLFLOW_SERVER_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.tensorflow.autolog()

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
mlflow.start_run(experiment_id = experiment.experiment_id)

<ActiveRun: >

# Preprocessing

In [15]:
def convert_to_rgb(image):
    if image.shape[-1] == 1:  # Vérifiez si l'image est en noir et blanc
        image = np.squeeze(image, axis=-1)  # Supprimez la dimension supplémentaire
        image = np.stack((image,) * 3, axis=-1)  # Dupliquez le canal pour en faire trois
    return image

In [16]:
# Configuration de l'ImageDataGenerator avec preprocessing_function
img_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalisation des pixels
    #preprocessing_function=convert_to_rgb,  # Fonction de conversion des cqnaux
    validation_split=0.2,  # Fraction pour la validation
    #rotation_range=90,
    #brightness_range=(0.5,1),
    #shear_range=0.2,
    #zoom_range=0.2,
    #channel_shift_range=0.2,
    #horizontal_flip=True,
    #vertical_flip=True
)

In [17]:
# Chargement des ensembles
BATCH_SIZE = 128
img_size = (256,256)
seed = 42

In [18]:
# Chargement des données  de train avec flow_from_directory
img_generator_flow_train = img_generator.flow_from_directory(
    directory="data/Main_dataset_7_classes",  # chemin vers les données
    target_size=img_size,    # taille des images
    batch_size=BATCH_SIZE,
    shuffle=True,
    color_mode='rgb',
    #color_mode="grayscale",    # pour charger les images en niveaux de gris
    subset="training",
    class_mode='categorical',
    seed = seed
    )

Found 44800 images belonging to 7 classes.


In [19]:
# Chargement des données  de validation avec flow_from_directory
img_generator_flow_valid = img_generator.flow_from_directory(
    directory="data/Main_dataset_7_classes",
    target_size=img_size,
    batch_size=BATCH_SIZE,
    shuffle=False,
    color_mode='rgb',
    #color_mode="grayscale",
    subset="validation",
    class_mode='categorical',
    seed= seed
    )

Found 11200 images belonging to 7 classes.


# Modèle

In [20]:
CLASSES = 7

In [21]:
#base_model = tf.keras.applications.EfficientNetB0(
#    input_shape=(128, 128, 3),
#    include_top=False,  # On enlève la tête de classification
#    weights="imagenet"  # Poids pré-entraînés sur ImageNet
#)

In [22]:
base_model = tf.keras.applications.VGG19(weights='imagenet',
                         include_top=False,
                         input_shape=(256, 256, 3),
                         name="vgg19"
                         )

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [23]:
base_model.trainable = False
#base_model.summary()

In [24]:
#model = tf.keras.Sequential([
#    base_model,  # Modèle EfficientNet
#    tf.keras.layers.GlobalAveragePooling2D(),
#    tf.keras.layers.Dense(128, activation='relu'),  # Couche dense
#    tf.keras.layers.Dropout(0.3),  # Régularisation
#    tf.keras.layers.Dense(CLASSES, activation='softmax')  # Classification binaire
#])

In [25]:
model = tf.keras.Sequential([
#    tf.keras.layers.Input(shape=(128, 128, 1)),  # Entrée en niveaux de gris
#    tf.keras.layers.Conv2D(3, (1, 1)),  # Conversion 1 -> 3 canaux
    base_model,
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES, activation="softmax")])


In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)                   │ (None, 8, 8, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8, 8, 128)           │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │          57,351 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,147,399 (76.86 MB)

 Trainable params: 123,015 (480.53 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

# Entrainement

In [27]:
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

In [28]:
# Compiler le modèle
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

In [29]:
EPOCHS = 20

In [30]:
# Entraîner le modèle avec early stopping
history = model.fit(
    #rgb_train_generator,
    img_generator_flow_train,
    validation_data=img_generator_flow_valid , #rgb_val_generator,
    epochs=EPOCHS ,  # Plus d'époques pour laisser l'early stopping décider
    #steps_per_epoch=len(img_generator_flow_train),
    #validation_steps=len(img_generator_flow_valid),
    #callbacks=[early_stopping],
    #class_weight= class_weights,
    #callbacks=[checkpoint]
    #callbacks = [reduce_lr]
)

2025/01/28 15:02:26 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
2025/01/28 15:02:26 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.2721 - loss: 1.8460

350/350 ━━━━━━━━━━━━━━━━━━━━ 949s 3s/step - categorical_accuracy: 0.2723 - loss: 1.8455 - val_categorical_accuracy: 0.4771 - val_loss: 1.4127
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.4976 - loss: 1.3627

350/350 ━━━━━━━━━━━━━━━━━━━━ 866s 2s/step - categorical_accuracy: 0.4977 - loss: 1.3626 - val_categorical_accuracy: 0.5462 - val_loss: 1.2440
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.5642 - loss: 1.1973

350/350 ━━━━━━━━━━━━━━━━━━━━ 869s 2s/step - categorical_accuracy: 0.5643 - loss: 1.1972 - val_categorical_accuracy: 0.5888 - val_loss: 1.1148
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.6077 - loss: 1.0872

350/350 ━━━━━━━━━━━━━━━━━━━━ 885s 2s/step - categorical_accuracy: 0.6077 - loss: 1.0871 - val_categorical_accuracy: 0.6313 - val_loss: 1.0268
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.6412 - loss: 1.0054

KeyboardInterrupt: 

# Sauvegarde du tracking MLFLOW

In [ ]:
mlflow.log_param("trainer", TRAINER)
mlflow.log_param("epochs", EPOCHS)
mlflow.log_param("model_type", MODEL_TYPE)

# Sauvegarde du modèle
mlflow.keras.log_model(model, "model")

# Sauvegarde des métriques par époque
history = model.history
for epoch in range(len(history.history['loss'])):
    mlflow.log_metric('loss', history.history['loss'][epoch], step=epoch)
for epoch in range(len(history.history['categorical_accuracy'])):
    mlflow.log_metric('categorical_accuracy', history.history['categorical_accuracy'][epoch], step=epoch)
for epoch in range(len(history.history['val_loss'])):
    mlflow.log_metric('val_loss', history.history['val_loss'][epoch], step=epoch)
for epoch in range(len(history.history['val_categorical_accuracy'])):
    mlflow.log_metric('val_categorical_accuracy', history.history['val_categorical_accuracy'][epoch], step=epoch)

In [ ]:
predictions = model.predict(img_generator_flow_valid)
y_pred = np.argmax(predictions, axis=1)
y_true = img_generator_flow_valid.classes

In [ ]:
report = classification_report(y_true, y_pred)
with open("classification_report.txt", "w") as file:
    file.write(report)
mlflow.log_artifact('classification_report.txt', artifact_path="model")

In [ ]:
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=img_generator_flow_valid.class_indices.keys())
disp.plot()
plt.title("Matrice de Confusion")
plt.xticks(rotation=90)
plt.savefig("confusion_matrix.png")
mlflow.log_artifact("confusion_matrix.png", artifact_path='model')

In [ ]:
report = classification_report(y_true, y_pred, output_dict=True)
mlflow.log_metric('global_accuracy', report['accuracy'])
mlflow.log_metric('macro_avg_precision', report['macro avg']['precision'])
mlflow.log_metric('macro_avg_recall', report['macro avg']['recall'])
mlflow.log_metric('macro_avg_f1_score', report['macro avg']['f1-score'])
mlflow.log_metric('macro_avg_support', report['macro avg']['support'])
mlflow.log_metric('weighted_avg_precision', report['weighted avg']['precision'])
mlflow.log_metric('weighted_avg_recall', report['weighted avg']['recall'])
mlflow.log_metric('weighted_avg_f1_score', report['weighted avg']['f1-score'])
mlflow.log_metric('weighted_avg_support', report['weighted avg']['support'])

In [ ]:
for class_mesure in list(report.items())[:CLASSES]:
    for m_name, m_value in class_mesure[1].items():
        mlflow.log_metric(m_name, m_value, step=int(class_mesure[0]))

In [ ]:
mlflow.end_run()